In [2]:
import torch
from loss import *

In [5]:
# Calculate the expected loss manually (using MSE loss)
mse_loss = nn.MSELoss(reduction="none")
masked_loss = MaskedLoss(mse_loss)


y_hat = torch.tensor(
    [
        [[0.9, 0.8], [0.7, 0.6], [0.5, 0.4], [0.3, 0.2], [0.1, 0.0]],
        [[0.5, 0.6], [0.7, 0.8], [0.9, 1.0], [1.1, 1.2], [1.3, 1.4]],
        [[1.0, 0.9], [0.8, 0.7], [0.6, 0.5], [0.4, 0.3], [0.2, 0.1]],
    ],
    dtype=torch.float32,
)
y = torch.tensor(
    [
        [[1.0, 0.8], [0.7, 0.6], [0.5, 0.4], [0.3, 0.2], [0.1, 0.0]],
        [[0.5, 0.6], [0.7, 0.8], [0.9, 1.0], [1.1, 1.2], [1.3, 1.4]],
        [[1.0, 0.9], [0.8, 0.7], [0.6, 0.5], [0.4, 0.3], [0.2, 0.1]],
    ],
    dtype=torch.float32,
)
mask = torch.tensor(
    [
        [1, 0, 1, 1, 0],
        [1, 1, 0, 1, 0],
        [1, 1, 1, 1, 1],
    ],
    dtype=torch.float32,
)

# Calculate the loss using the MaskedLoss class
masked_loss = masked_loss.get_loss(y_hat, y, mask)
expected_loss = mse_loss(y_hat, y)

In [6]:
masked_loss

tensor(0.0005)

In [8]:
torch.sum(expected_loss)

tensor(0.0100)

In [1]:
import numpy as np

In [283]:
def calculate_directional_consistency(trajectory):
    """
    Calculate directional consistency of a 3D trajectory.

    Args:
        trajectory (numpy.ndarray): NumPy array of shape (num_frame, 3) representing the 3D trajectory.

    Returns:
        float: Variance of the angles between consecutive direction vectors.
    """
    if trajectory.shape[0] < 2:
        raise ValueError("Trajectory must have at least two data points for directional consistency calculation.")

    # Compute direction vectors between consecutive points
    direction_vectors = np.diff(trajectory, axis=0)

    # Calculate angles between consecutive direction vectors
    angles = []
    for i in range(1, len(direction_vectors)):
        v1 = direction_vectors[i - 1]
        v2 = direction_vectors[i]
        cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
        angle = np.arccos(np.clip(cos_theta, -1.0, 1.0))
        angles.append(angle)

    # Calculate variance of the angles
    variance = np.var(angles)

    return variance

# Example usage:
if __name__ == "__main__":
    # Create a sample 3D trajectory (num_frame, 3)
    sample_trajectory = np.array([[0.0, 0.0, 0.0],
                                   [1.0, 0.0, 0.0],
                                   [2.0, 0.0, 0.0],
                                   [2.0, 1.0, 0.0],
                                   [2.0, 2.0, 0.0]])
    
    sample_trajectory = np.random.rand(50, 3)

    # Calculate directional consistency
    variance = calculate_directional_consistency(sample_trajectory)

    print(f"Directional consistency (variance of angles): {variance}")


Directional consistency (variance of angles): 0.3138348994231784
